In [ ]:
lab_enc = True
ohe = False
new_features = True
hp_tune = False
balance_train = False
add_tags = False

In [ ]:
tags = []

## Column Names and Description

In [ ]:
"""
Columns
The following columns are available on the training set:

accepts_mercadopago Whether the item accepts Mercado Pago
available_quantity The available stock quantity at that moment
avg_asp_item_bday Average selling price of this item over the last days
avg_asp_item_domain Average price of items of the domain this item belongs to
avg_asp_item_sel Average price of seller sales
avg_gmv_item_bday average revenue generated by the item per day
avg_gmv_item_domain_30days Average revenue generated by the items of this domain on the last month
avg_gmv_item_sel Average revenue of items of this seller
avg_gmv_seller_bday Average revenue this seller makes by day
avg_orders_item_bday Average number of orders this item has by day
avg_orders_seller_bday Average orders the seller has by day
avg_qty_orders_item_domain_30days Average number of orders a random item of this domain made on the last month
avg_qty_orders_item_sel_30days Average number of orders an item of this seller makes on the last 30 days
avg_si_item_bday Average units sold this item has by day
avg_si_item_sel_30day Average units sold of an item of this seller on the past month
avg_visits_item Average visits this item had
benefit Ignore, should be dropped
boosted Whether the item was boosted
buy_server_timestamp Timestamp of the purchase
category_id Category of this item
conversion Target variable, it is True if this print has an attributed order
cus_cust_id Buyer id
cus_cust_id_sel Seller id
date Print date
deal_print_id Unique id for the print
decimals Ignore, will be dropped
domain_id Domain id for the item
etl_version Ignore, should be dropped
free_shipping Whether the item has free shipping
fulfillment Whether the item is fulfilled by MeLi
full_name Category full name
health Item health
is_pdp Whether the click landed on a PDP
product_id Product_id of the item
item_id ID of the item, useful for debugging
listing_type_id Whether the item is gold or not
logistic_type Logistic type for the item
main_picture URL for the main item picture
offset On which page the item was rendered
original_price Price from which the discount was done
platform Which platform the user is using
price Item price
print_position Position on the page
print_server_timestamp Timestamp for the print
qty_items_dom Number of items this domain has
qty_items_sel Number of items the seller has
rn Leftover from the ETL, Discard
ROW_ID Row of the submission file
site_id Site ID
sold_quantity Number of items sold at the moment of the print
tags Tags the item had at the moment of the print
title Item title
total_asp_item_domain_30days Average selling price of the items of the domain
total_asp_item_sel_30days Average selling price of all the items the seller sold on the last 30 days
total_gmv_domain_30days Total revenue the domain made on the last 30 days
total_gmv_domain_bday total_gmv_domain_30days / 30
total_gmv_item_30days Total revenue made by the item on the lasts 30 days
total_gmv_seller Total revenue made by the seller on the last month
total_items_domain Number of items on the domain
total_items_seller Number of items the seller has
total_orders_domain_30days Total orders on the domain
total_orders_item_30days Total orders the Item had on the last 30 days
total_orders_sel_30days Total orders for the seller
total_si_domain_30days Total units sold of this domain
total_si_item_30days Total units sold of this item
total_si_sel_30days Same for the seller
total_visits_domain Total visits on this domain
total_visits_item Total visits this item had
total_visits_seller Total visits for this seller
uid session id
user_id user id
warranty Whether the item had warranty
"""

## Librerias

In [ ]:
from IPython.display import display, Markdown

import pandas as pd
import gc
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier, HistGradientBoostingClassifier

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold, cross_val_score

from sklearn.metrics import roc_auc_score

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

from sklearn.tree import plot_tree

from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler, OneHotEncoder

from sklearn.metrics import roc_curve, auc

# XGBoost
import xgboost as xgb

import numpy as np

import matplotlib.pyplot as plt

import itertools

from tqdm import tqdm

## Funciones

In [ ]:
def plot_roc_curves(models, model_names, X_test, y_test, ax=None):

    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 10))

    for i in range(len(models)):
        fpr, tpr, thresholds = roc_curve(y_test, models[i].predict_proba(X_test)[:, models[i].classes_ == 1])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=1, label=model_names[i] + ' (area = %0.5f)' % roc_auc)

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

    plt.xlim([-0.05, 1.0])

    plt.ylim([0.0, 1.05])

    plt.xlabel('False Positive Rate')

    plt.ylabel('True Positive Rate')

    plt.title('Receiver operating characteristic example')

    # Show grid
    plt.grid(True)

    plt.legend(loc="lower right")

    plt.show()

## Load Comp Data

In [ ]:
comp_data = pd.read_csv("data/competition_data.csv")

In [ ]:
comp_data.head()

In [ ]:
comp_data.describe()

In [ ]:
display(
    Markdown(
        "\n".join(
            [
                #"- **{}** ({}) \n\n {}".format(col, dtype, comp_data[col].value_counts()) for col, dtype in
                #"- **{}** ({})".format(col, dtype) for col, dtype in
                #zip(comp_data.columns, comp_data.dtypes)
                "- **{}**".format(col) for col in comp_data.columns
            ]
        )
    )
)

## Data Cleaning

In [ ]:
comp_data = pd.read_csv("data/competition_data.csv")
comp_data = comp_data.drop(
        columns=[
            "accepts_mercadopago", # Todos tienen mercadopago
            "boosted", # Ninguno
            "avg_qty_orders_item_domain_30days",
            "avg_qty_orders_item_sel_30days",
            "avg_si_item_sel_30day",
            #"date",
            "deal_print_id",
            "etl_version",
            "full_name",
            "main_picture",
            "print_server_timestamp",
            #"title",
            "uid",
            #"user_id",
            "avg_gmv_item_domain_30days",
            #"avg_gmv_item_sel",
            "avg_gmv_seller_bday",
            #"qty_items_dom",
            #"category_id",
            #"domain_id",
            #"product_id",
            #"listing_type_id",
            # "logistic_type",
            "site_id",
            #"tags",
            "total_asp_item_sel_30days", 
            "total_gmv_domain_bday", 
            #"total_items_domain", 
            "total_items_seller", 
            "total_orders_domain_30days", 
            "total_orders_item_30days", 
            "total_orders_sel_30days", 
            "total_si_sel_30days", 
            #"total_visits_domain", 
            #"total_visits_item", 
            #"total_visits_seller",
            "total_asp_item_domain_30days",
            "total_gmv_item_30days",
            "total_si_domain_30days",
            "total_si_item_30days",
            "warranty",
            # "offset",
        ]
    )

In [ ]:
display(
    Markdown(
        "\n".join(
            [
                #"- **{}** ({}) \n\n {}".format(col, dtype, comp_data[col].value_counts()) for col, dtype in
                #"- **{}** ({})".format(col, dtype) for col, dtype in
                #zip(comp_data.columns, comp_data.dtypes)
                "- **{}**".format(col) for col in comp_data.columns
            ]
        )
    )
)

In [ ]:
if add_tags:
    if len(tags) == 0:
        tags = comp_data["tags"].str.replace("[", "").str.replace("]", "").str.split(", ").apply(pd.Series).stack().value_counts()

In [ ]:
# tags

In [ ]:
# # For tag in tags, create a new column with the tag name and a boolean value

if add_tags:
    for tag in tags.index:
        comp_data[tag] = comp_data["tags"].str.contains(tag).astype(int)

    comp_data["tags_count"] = comp_data["tags"].str.replace("[", "").str.replace("]", "").str.split(", ").apply(len)

    comp_data = comp_data.drop("tags", axis=1)

In [ ]:
# comp_data["domain_id"] = LabelEncoder().fit_transform(comp_data["domain_id"]).astype(int)
# comp_data["product_id"] = LabelEncoder().fit_transform(comp_data["product_id"]).astype(int)

# OneHotEncoder
# comp_data = pd.get_dummies(comp_data, columns=["category_id"], dtype=int, sparse=True, dummy_na = False)
# Add category_id ohe category column if that category has more than 1000 ocurrences
# ocurrences = comp_data["category_id"].value_counts()
# for category_id in ocurrences[ocurrences > 1000].index:
#     comp_data["category_id_" + str(category_id)] = comp_data["category_id"] == category_id
#     comp_data["category_id_" + str(category_id)] = comp_data["category_id_" + str(category_id)].astype(int)
# comp_data = comp_data.drop(columns=["category_id"])

# comp_data = pd.get_dummies(comp_data, columns=["logistic_type"], dtype=int, sparse=True, dummy_na = False)

In [ ]:
comp_data["date"] = pd.to_datetime(comp_data["date"])
comp_data["day"] = comp_data["date"].dt.day
comp_data["month"] = comp_data["date"].dt.month
#comp_data["hour"] = comp_data["date"].dt.hour
# comp_data["year"] = comp_data["date"].dt.year
comp_data["dayofweek"] = comp_data["date"].dt.dayofweek
# comp_data["weekofyear"] = comp_data["date"].dt.isocalendar().week
#comp_data["quarter"] = comp_data["date"].dt.quarter
# comp_data["hour"] = comp_data["date"].dt.hour
# comp_data["minute"] = comp_data["date"].dt.minute


In [ ]:
comp_data["date"].min(), comp_data["date"].max()

In [ ]:
comp_data = comp_data.drop(columns=["date"])

### Parse boolean columns

In [ ]:
# comp_data["boosted"] = comp_data["boosted"].astype(int)
comp_data["free_shipping"] = comp_data["free_shipping"].astype(int)
comp_data["fulfillment"] = comp_data["fulfillment"].astype(int)

comp_data["imp_is_pdp"] = comp_data["is_pdp"].isna().astype(int)
comp_data["is_pdp"].fillna(0, inplace=True)
comp_data["is_pdp"] = comp_data["is_pdp"].astype(int)

comp_data["imp_user_id"] = comp_data["user_id"].isna().astype(int)
comp_data["user_id"] = comp_data["user_id"].fillna(0).astype(int)

comp_data["listing_type_id"] = comp_data["listing_type_id"].apply(lambda x: 0 if x == "gold_special" else 1)

### Encoding

In [ ]:
if lab_enc:
    # LabelEncoder
    comp_data["platform"] = LabelEncoder().fit_transform(comp_data["platform"]).astype(int)
    #comp_data["category_id"] = LabelEncoder().fit_transform(comp_data["category_id"]).astype(int)
    comp_data["product_id"] = LabelEncoder().fit_transform(comp_data["product_id"]).astype(int)
    comp_data["domain_id"] = LabelEncoder().fit_transform(comp_data["domain_id"]).astype(int)
    comp_data["logistic_type"] = LabelEncoder().fit_transform(comp_data["logistic_type"]).astype(int)

if ohe:
    # OHE
    comp_data = pd.get_dummies(comp_data,
                            columns = [
                                # "platform",
                                "category_id",
                                # "domain_id"
                                ],
                            sparse = True,    # Devolver una matriz rala.
                            dummy_na = False, # No agregar columna para NaNs.
                            dtype = int       # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                        )


# Encode category_id as a number, but number is based on ocurrences of category_id
# ! Has data leakage
#ocurrences = comp_data["category_id"].value_counts()
#for category_id in ocurrences.index:
#    comp_data["category_id"] = comp_data["category_id"].replace(category_id, ocurrences[category_id])

# comp_data["category_id"] = comp_data["category_id"].apply(lambda x: ocurrences[x])

### New columns

In [ ]:
if new_features:
    comp_data["price_diff"] = comp_data["price"] - comp_data["original_price"]
    comp_data["price_diff"] = comp_data["price_diff"].apply(lambda x: abs(x)).astype(int)

    comp_data["discount"] = (comp_data["original_price"] - comp_data["price"]) / comp_data["original_price"]
    comp_data["discount"] = comp_data["discount"].apply(lambda x: 1 if x == np.inf else x)

    comp_data["price_ratio"] = comp_data["price"] / comp_data["original_price"]
    comp_data["price_ratio"] = comp_data["price_ratio"].apply(lambda x: 1 if x == np.inf else x)

    comp_data["is_discount"] = comp_data["discount"].apply(lambda x: 1 if x > 0 else 0)

    comp_data["title_length"] = comp_data["title"].str.len()
    comp_data["title_word_count"] = comp_data["title"].str.split(" ").apply(len)

    comp_data["title_length_word_count"] = comp_data["title_length"] / comp_data["title_word_count"]
    comp_data["title_length_word_count"] = comp_data["title_length_word_count"].apply(lambda x: 1 if x == np.inf else x)

    comp_data["domain_dominance"] = comp_data["sold_quantity"] / comp_data["qty_items_dom"]

    comp_data["is_pdp_tvi"] = comp_data["is_pdp"] / comp_data["total_visits_item"]
    comp_data["is_pdp_tvi"] = comp_data["is_pdp_tvi"].apply(lambda x: 1 if x == np.inf else x)

    comp_data["is_pdp_tvs"] = comp_data["is_pdp"] / comp_data["total_visits_seller"]
    comp_data["is_pdp_tvs"] = comp_data["is_pdp_tvs"].apply(lambda x: 1 if x == np.inf else x)

    comp_data["is_pdp_tvd"] = comp_data["is_pdp"] / comp_data["total_visits_domain"]
    comp_data["is_pdp_tvd"] = comp_data["is_pdp_tvd"].apply(lambda x: 1 if x == np.inf else x)


In [ ]:
# comp_data = comp_data.dropna(subset=["avg_gmv_item_sel"])

# Fil nas for ["avg_gmv_item_sel", "qty_items_dom", "qty_items_sel", "domain_dominance"]

comp_data["nan_imputed"] = comp_data["avg_gmv_item_sel"].isna().astype(int)

comp_data["avg_gmv_item_sel"] = comp_data["avg_gmv_item_sel"].fillna(comp_data["avg_gmv_item_sel"].median())
comp_data["qty_items_dom"] = comp_data["qty_items_dom"].fillna(comp_data["qty_items_dom"].median())
comp_data["qty_items_sel"] = comp_data["qty_items_sel"].fillna(comp_data["qty_items_sel"].median())
comp_data["domain_dominance"] = comp_data["domain_dominance"].fillna(comp_data["domain_dominance"].median())

## Exploratory

In [ ]:
display(
    Markdown(
        "\n".join(
            [
                #"- **{}** ({}) \n\n {}".format(col, dtype, comp_data[col].value_counts()) for col, dtype in
                #"- **{}** ({})".format(col, dtype) for col, dtype in
                #zip(comp_data.columns, comp_data.dtypes)
                "- **{}**".format(col) for col in comp_data.columns
            ]
        )
    )
)

In [ ]:
# Check this columns for variance
# accepts_mercadopago
# available_quantity
# avg_gmv_item_domain_30days
# avg_gmv_item_sel
# avg_gmv_seller_bday
# avg_qty_orders_item_domain_30days
# avg_qty_orders_item_sel_30days
# avg_si_item_sel_30day
# benefit
# boosted
# category_id
# conversion
# date
# deal_print_id
# domain_id
# etl_version
# free_shipping
# fulfillment
# full_name
# health
# is_pdp
# product_id
# item_id
# listing_type_id
# logistic_type
# main_picture
# offset
# original_price
# platform
# price
# print_position
# print_server_timestamp
# qty_items_dom
# qty_items_sel
# old_quantity
# tags
# title
# total_asp_item_domain_30days
# total_asp_item_sel_30days
# total_gmv_domain_bday
# total_gmv_item_30days
# total_items_domain
# total_items_seller
# total_orders_domain_30days
# total_orders_item_30days
# total_orders_sel_30days
# total_si_domain_30days
# total_si_item_30days
# total_si_sel_30days
# total_visits_domain
# total_visits_item
# total_visits_seller
# uid
# user_id
# warranty

# for col in comp_data.columns:
#     print()
#     print(col)
#     print(comp_data[col].value_counts().head(5))

In [ ]:
# comp_data.groupby("user_id")["conversion"].sum()
# comp_data[comp_data["conversion"] == True]["user_id"].value_counts()

In [ ]:
# Balance conversion
# comp_data = comp_data.drop(comp_data[comp_data["conversion"] == False].sample(frac=.5).index)

In [ ]:
# comp_data["conversion"].value_counts()

### Plots

In [ ]:
# Plot 'offset' vs 'category_id' . Dot color depends on 'conversion'

plt.scatter(full_data["offset"], full_data["discount"], c=full_data["conversion"])

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(
    new_train_data["offset"],
    new_train_data["discount"],
    new_train_data["is_pdp"],
    c=new_train_data["conversion"],
    # cmap="coolwarm",
    alpha=.5
)

ax.set_xlabel("offset")
ax.set_ylabel("discount")
ax.set_zlabel("is_pdp")


In [ ]:
# # Plot 'offset', 'discount', 'category_id' in 3D. Dot color depends on 'conversion'

# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter(
#     new_test_data["offset"],
#     new_test_data["discount"],
#     new_test_data["price_diff"],
#     c=new_test_data["conversion"],
#     # cmap="coolwarm",
#     alpha=.5
# )

# ax.set_xlabel("offset")
# ax.set_ylabel("discount")
# ax.set_zlabel("price_diff")


In [ ]:
# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(111, projection='3d')
# # ax = fig.add_subplot(111)
# ax.scatter(
#     new_test_data["offset"],
#     new_test_data["price_diff"],
#     new_test_data["offset"] ** 2 + new_test_data["price_diff"]**2,
#     c=new_test_data["conversion"],
# )

# plt.show()

In [ ]:
# # Plot hist boosted
# plot_data = comp_data.groupby("is_pdp")["conversion"].sum()
# plot_data


In [ ]:
# Plot hour vs conversion
#comp_data.groupby("dayofweek")["conversion"].mean().plot()

## Splits

### Split Comp Data -> Full | Eval

In [ ]:
full_data = comp_data[comp_data["ROW_ID"].isna()]
eval_data = comp_data[comp_data["ROW_ID"].notna()]
# del comp_data
gc.collect()

In [ ]:
# df = pd.read_csv("data/competition_data.csv")
# print(df.isna().sum()[df.isna().sum() > 0])
# del df
# gc.collect()

In [ ]:
# Count nan values per column

full_data.isna().sum()[full_data.isna().sum() > 0]

In [ ]:
# The following cols have nan values
# avg_gmv_item_sel
# qty_items_dom
# qty_items_sel
# domain_dominance depends on qty_items_dom

# For those columns, are the rows with nan values the same?

# full_data[full_data["avg_gmv_item_sel"].isna()]["qty_items_dom"].isna().sum(), full_data[full_data["avg_gmv_item_sel"].isna()]["qty_items_sel"].isna().sum(), full_data[full_data["avg_gmv_item_sel"].isna()]["domain_dominance"].isna().sum()

# Yes, they are the same rows

In [ ]:
# Drop those rows

# full_data = full_data.dropna(subset=["avg_gmv_item_sel"])

In [ ]:
# Count nan values per column

full_data.isna().sum()[full_data.isna().sum() > 0]

### Train Test Split

In [ ]:
# Shuffle
full_data = full_data.sample(frac=1, random_state=19092140).reset_index(drop=True)

train_data, test_data = train_test_split(full_data, test_size=0.25, train_size=0.75, random_state=42)

### Balancing

In [ ]:
count_converts_test = test_data[test_data["conversion"] == True].shape[0]
count_not_converts_test = test_data[test_data["conversion"] == False].shape[0]
ratio = count_converts_test / count_not_converts_test

# From test_data, keep ratio of count_not_converts_test entries

not_converts = test_data[test_data["conversion"] == False].sample(frac=ratio)

new_test_data = pd.concat([test_data[test_data["conversion"] == True], not_converts])
new_test_data["conversion"].value_counts()

In [ ]:
# Balance Train Data by conversion

len(train_data[train_data["conversion"] == False])

In [ ]:
if balance_train:

    count_converts_train = len(train_data[train_data["conversion"] == True])
    count_not_converts_train = len(train_data[train_data["conversion"] == False])

    ratio = count_converts_train / count_not_converts_train

    # From train_data, keep ratio of count_not_converts_train entries

    not_converts = train_data[train_data["conversion"] == False].sample(frac=ratio)

    new_train_data = pd.concat([train_data[train_data["conversion"] == True], not_converts])

    train_data = new_train_data

In [ ]:
# ! Oversample, repeats into Validation
# train_data = train_data.append(train_data[train_data["conversion"] == True].sample(frac=1, replace=True))
# train_data = train_data.append(train_data[train_data["conversion"] == True].sample(frac=1, replace=True))
# train_data["conversion"].value_counts()

###  Split X | y

In [ ]:
y_train = train_data["conversion"]
X_train = train_data.drop(columns=["conversion", "ROW_ID"])
X_train = X_train.select_dtypes(include='number')

y_test = test_data["conversion"]
X_test = test_data.drop(columns=["conversion", "ROW_ID"])
X_test = X_test.select_dtypes(include='number')

del train_data
del test_data

gc.collect()

### Validation Split

In [ ]:
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.25, train_size=0.75, random_state=42)

## Models

### XGBoost

In [ ]:
random_state = 12345

### Train XGBoost Model (Default params) with Cross Validation

In [ ]:
xgb1_model = xgb.XGBClassifier(
    objective = 'binary:logistic',
    seed = random_state,
    eval_metric = 'auc',
    n_jobs=-1,
    #base_score=0.74
)

xgb1_model.fit(X_train_sub, y_train_sub, eval_set=[(X_val, y_val)], verbose=20)

#### ROC AUC

In [ ]:
roc_auc_score(y_test, xgb1_model.predict_proba(X_test)[:, xgb1_model.classes_ == 1])

In [ ]:
# Plot ROC curve

fpr, tpr, thresholds = roc_curve(y_test, xgb1_model.predict_proba(X_test)[:, xgb1_model.classes_ == 1])
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 10))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.5f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

plt.xlim([0.0, 1.0])

plt.ylim([0.0, 1.05])

plt.xlabel('False Positive Rate')

plt.ylabel('True Positive Rate')

plt.title('Receiver operating characteristic example')

plt.legend(loc="lower right")

plt.show()

In [ ]:
# Feature Importance
xgb.plot_importance(xgb1_model, max_num_features=20, importance_type="gain", xlabel="Gain")

In [ ]:
# Feature Importance
xgb.plot_importance(xgb1_model, max_num_features=20, importance_type="cover", xlabel="Cover")

#### Feature Importance

In [ ]:
# Get Feature Importance from the classifier xgb1_model
feature_importances = pd.DataFrame(
    xgb1_model.feature_importances_,
    index = X_train.columns,
    columns=['importance']).sort_values('importance', ascending=False)

feature_importances.head(20)

In [ ]:
# Add gain column
gains = sorted(xgb1_model.get_booster().get_score(importance_type="gain").items(), key=lambda x: x[1], reverse=True)

gains_pd = pd.DataFrame(gains, columns=["feature", "gain"]).set_index("feature").sort_values("gain", ascending=False)

gains_pd.head(20)

In [ ]:
feature_importances.head(20)

### Train XGBoost Model (Top 20 Gain params) with Cross Validation

In [ ]:
xgb2_model = xgb.XGBClassifier(
    objective = 'binary:logistic',
    seed = random_state,
    n_jobs=-1,
)

reduced_cols = gains_pd.head(20).index

xgb2_model.fit(X_train, y_train)

In [ ]:
roc_auc_score(y_test, xgb2_model.predict_proba(X_test)[:, xgb2_model.classes_ == 1])

In [ ]:
covers = sorted(xgb1_model.get_booster().get_score(importance_type="cover").items(), key=lambda x: x[1], reverse=True)
covers_pd = pd.DataFrame(covers, columns=["feature", "cover"]).set_index("feature").sort_values("cover", ascending=False)


xgb_model_covers = xgb.XGBClassifier(
    objective = 'binary:logistic',
    seed = random_state,
    n_jobs=-1,
)

reduced_cols = covers_pd.head(20).index

xgb_model_covers.fit(X_train[reduced_cols], y_train)

roc_auc_score(y_test, xgb_model_covers.predict_proba(X_test[reduced_cols])[:, xgb_model_covers.classes_ == 1])

#### ROC AUC

In [ ]:
roc_auc_score(y_test, xgb2_model.predict_proba(X_test)[:, xgb2_model.classes_ == 1])

#### Feature Importance

In [ ]:
# Feature Importance
xgb.plot_importance(xgb2_model, max_num_features=20, importance_type="gain", xlabel="Gain")

In [ ]:
# Feature Importance
fig, ax = plt.subplots(figsize=(5, 20))

xgb.plot_importance(xgb2_model, max_num_features=100, importance_type="gain", xlabel="Gain", ax=ax)

## Retrain Model with Validation Data

In [ ]:
# del comp_data
# del full_data
# gc.collect()

In [ ]:
# params = {'colsample_bytree': 0.9958188825321641, 'gamma': 0.9488078042954764, 'learning_rate': 0.07108927605766976, 'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 4, 'subsample': 0.8232243201689888}
xgb4_model = xgb.XGBClassifier(
        # **params,
        objective = 'binary:logistic',
        seed = random_state,
        n_jobs=-1,
)

xgb4_model.fit(X_train, y_train)

#### ROC AUC on X_test

In [ ]:
roc_auc_score(y_test, xgb4_model.predict_proba(X_test)[:, xgb4_model.classes_ == 1])

#### ROC AUC on Balanced X_test (Subsampled X_test for conversion == 0)

In [ ]:
new_X_test = new_test_data.drop(columns=["conversion", "ROW_ID"])
new_X_test = new_X_test.select_dtypes(include='number')

new_y_test = new_test_data["conversion"]

roc_auc_score(new_y_test, xgb2_model.predict_proba(new_X_test)[:, xgb2_model.classes_ == 1])

#### Confusion Matrix

In [ ]:
# Confusion Matrix for xgb2_model on new_test_data

from sklearn.metrics import confusion_matrix

conf_mx = confusion_matrix(new_y_test, xgb2_model.predict(new_X_test))

true_neg, false_pos, false_neg, true_pos = conf_mx.ravel()

true_neg, false_pos, false_neg, true_pos

In [ ]:
""" xgb3_model = make_pipeline(
    SimpleImputer(strategy="median"),
    xgb.XGBClassifier(
        objective = 'binary:logistic',
        seed = random_state,
        n_jobs=-1,
    )
)

xgb3_model.fit(X_train, y_train)

roc_auc_score(y_test, xgb3_model.predict_proba(X_test)[:, xgb3_model.classes_ == 1]) """

### Random Forest

In [ ]:
rfc = make_pipeline(
    SimpleImputer(strategy="median"),
    RandomForestClassifier()
)

rfc.fit(X_train, y_train)

roc_auc_score(y_test, rfc.predict_proba(X_test)[:, rfc.classes_ == 1])

### Voting Classifier (XGBoost + Random Forest)

In [ ]:
voting_rfc_xgb = VotingClassifier(
    estimators=[
        ("rfc", rfc),
        ("xgb", xgb4_model)
    ],
    voting="soft",
    n_jobs=-1
)

voting_rfc_xgb.fit(X_train, y_train)

roc_auc_score(y_test, voting_rfc_xgb.predict_proba(X_test)[:, voting_rfc_xgb.classes_ == 1])

### GradientBoostingClassifier

In [ ]:
gbc = make_pipeline(
    SimpleImputer(strategy="median"),
    GradientBoostingClassifier()
)

gbc.fit(X_train, y_train)

roc_auc_score(y_test, gbc.predict_proba(X_test)[:, gbc.classes_ == 1])

### HistGradientBoostingClassifier

In [ ]:
hgb = HistGradientBoostingClassifier()

hgb.fit(X_train, y_train)

roc_auc_score(y_test, hgb.predict_proba(X_test)[:, hgb.classes_ == 1])

### AdaBoostClassifier

In [ ]:
abc = make_pipeline(
    SimpleImputer(strategy="median"),
    AdaBoostClassifier()
)

abc.fit(X_train, y_train)

roc_auc_score(y_test, abc.predict_proba(X_test)[:, abc.classes_ == 1])

### ExtraTreesClassifier

In [ ]:
etc = make_pipeline(
    SimpleImputer(strategy="median"),
    ExtraTreesClassifier()
)

etc.fit(X_train, y_train)

roc_auc_score(y_test, etc.predict_proba(X_test)[:, etc.classes_ == 1])

### Voting Classifier Alt

In [ ]:
voting_alt = VotingClassifier(
    estimators=[
        ("rfc", rfc),
        ("xgb", xgb4_model),
        ("hgb", hgb),
        ("abc", abc),
    ],
    voting="soft",
    n_jobs=-1
)

voting_alt.fit(X_train, y_train)

roc_auc_score(y_test, voting_alt.predict_proba(X_test)[:, voting_alt.classes_ == 1])

In [ ]:
voting_alt

### Average Classifier

In [ ]:
# average classifier
from sklearn.utils._pprint import _EstimatorPrettyPrinter

class AverageClassifier:
    def __init__(self, models):
        self.models = models
        self.classes_ = models[0].classes_

    def predict_proba(self, X):
        return np.mean([model.predict_proba(X) for model in self.models], axis=0)

    def predict(self, X):
        return np.mean([model.predict(X) for model in self.models], axis=0)

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)

    def __repr__(self, N_CHAR_MAX=700):
        # N_CHAR_MAX is the (approximate) maximum number of non-blank
        # characters to render. We pass it as an optional parameter to ease
        # the tests.

        # Using _EstimatorPrettyPrinter

        pp = _EstimatorPrettyPrinter(
            compact=True,
            indent=1,
            indent_at_name=True,
            n_max_elements_to_show=N_MAX_ELEMENTS_TO_SHOW,
        )

        repr_ = pp.pformat(self)

        return repr_
        


average_classifier = AverageClassifier([rfc, xgb4_model, hgb, abc, voting_alt])

# average_classifier.fit(X_train, y_train)

roc_auc_score(y_test, average_classifier.predict_proba(X_test)[:, average_classifier.classes_ == 1])

### Stacking Classifier

In [ ]:
# Stacking Classifier

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

stacking_classifier = StackingClassifier(
    estimators=[
        ("rfc", rfc),
        ("xgb", xgb4_model),
        ("hgb", hgb),
        ("abc", abc),
    ],
    final_estimator=LogisticRegression(),
    n_jobs=-1
)

stacking_classifier.fit(X_train, y_train)

roc_auc_score(y_test, stacking_classifier.predict_proba(X_test)[:, stacking_classifier.classes_ == 1])

In [ ]:
stacking_classifier

In [ ]:
average_classifier2 = AverageClassifier([voting_alt, stacking_classifier])

average_classifier.fit(X_train, y_train)

roc_auc_score(y_test, average_classifier2.predict_proba(X_test)[:, average_classifier2.classes_ == 1])

### Logistic Regression

In [ ]:
# Logistic Regression

logistic_regression = make_pipeline(
    SimpleImputer(strategy="median"),
    LogisticRegression(
        n_jobs=-1
    )
)

best_score = 0
best_cut = 0
for i in range(1, 20):

    logistic_regression.fit(X_train[reduced_cols[:i]], y_train)

    score = roc_auc_score(y_test, logistic_regression.predict_proba(X_test[reduced_cols[:i]])[:, logistic_regression.classes_ == 1])

    print(i, score)

    if score > best_score:
        best_score = score
        best_cut = i

best_score, best_cut

In [ ]:
logistic_regression = make_pipeline(
    SimpleImputer(strategy="median"),
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
    LogisticRegression(
        n_jobs=-1,
        max_iter=1000
    )
)

logistic_regression.fit(X_train[reduced_cols[:3]], y_train)

roc_auc_score(y_test, logistic_regression.predict_proba(X_test[reduced_cols[:3]])[:, logistic_regression.classes_ == 1])

In [ ]:
from sklearn.ensemble import BaggingClassifier


In [ ]:
bagging_classifier_rfc = make_pipeline(
    SimpleImputer(strategy="median"),
    BaggingClassifier(
        base_estimator=RandomForestClassifier(),
        n_estimators=10,
        n_jobs=-1
    )
)

bagging_classifier_rfc.fit(X_train, y_train)

roc_auc_score(y_test, bagging_classifier_rfc.predict_proba(X_test)[:, bagging_classifier_rfc.classes_ == 1])

In [ ]:
bagging_classifier_logistic = BaggingClassifier(
    estimator=LogisticRegression(
        n_jobs=-1,
        max_iter=1000
    ),
    n_estimators=10,
    n_jobs=-1
)

bagging_classifier_logistic.fit(X_train[reduced_cols[:best_cut]], y_train)

roc_auc_score(y_test, bagging_classifier_logistic.predict_proba(X_test[reduced_cols[:best_cut]])[:, bagging_classifier_logistic.classes_ == 1])

In [ ]:
bagging_classifier_xgb = BaggingClassifier(
    base_estimator=xgb.XGBClassifier(
        objective = 'binary:logistic',
        seed = random_state,
        n_jobs=-1,
    ),
    n_estimators=10,
    n_jobs=-1
)

bagging_classifier_xgb.fit(X_train, y_train)

roc_auc_score(y_test, bagging_classifier_xgb.predict_proba(X_test)[:, bagging_classifier_xgb.classes_ == 1])

In [ ]:
stacking_classifier_2 = StackingClassifier(
    estimators=[
        ("rfc", rfc),
        ("xgb", xgb4_model),
        ("hgb", hgb),
        ("abc", abc),
        ("bg_xgb", bagging_classifier_xgb),
    ],
    final_estimator=LogisticRegression(),
    n_jobs=-1
)

stacking_classifier_2.fit(X_train, y_train)

roc_auc_score(y_test, stacking_classifier_2.predict_proba(X_test)[:, stacking_classifier_2.classes_ == 1])

In [ ]:
voting_alt_2 = VotingClassifier(
    estimators=[
        ("rfc", rfc),
        ("xgb", xgb4_model),
        ("hgb", hgb),
        ("abc", abc),
        ("bg_xgb", bagging_classifier_xgb),
    ],
    voting="soft",
    n_jobs=-1
)

voting_alt_2.fit(X_train, y_train)

roc_auc_score(y_test, voting_alt_2.predict_proba(X_test)[:, voting_alt_2.classes_ == 1])

In [ ]:
average_classifier_3 = AverageClassifier([voting_alt_2, stacking_classifier_2])

average_classifier_3.fit(X_train, y_train)

roc_auc_score(y_test, average_classifier_3.predict_proba(X_test)[:, average_classifier_3.classes_ == 1])

In [ ]:
y_train[y_train == True].shape[0], y_train[y_train == False].shape[0]

#### ROC AUC Curves

In [ ]:
models = [
    #(logistic_regression, "logistic_regression"),
    (rfc, "RandomForestClassifier"),
    (xgb1_model, "xgb1_model"),
    (xgb2_model, "xgb2_model"),
    (xgb4_model, "xgb4_model"), 
    (voting_rfc_xgb, "voting_rfc_xgb"), 
    #(gbc, "gbc"), 
    (hgb, "hgb"),
    (abc, "abc"), 
    # (etc, "etc"), 
    (voting_alt, "voting_alt"),
    (voting_alt_2, "voting_alt_2"),
    (stacking_classifier, "stacking_classifier"),
    (stacking_classifier_2, "stacking_classifier_2"),
    (average_classifier, "average_classifier"),
    (average_classifier2, "average_classifier2"),
    (average_classifier_3, "average_classifier_3"),
    # (bagging_classifier_rfc, "bagging_classifier_rfc"),
    # (bagging_classifier_logistic, "bagging_classifier_logistic"),
    (bagging_classifier_xgb, "bagging_classifier_xgb"),
]

model_names = [model[1] for model in models]
models = [model[0] for model in models]

fig, ax = plt.subplots(figsize=(10, 10))

# plot_roc_curves(models, model_names, new_X_test, new_y_test, ax=ax)
plot_roc_curves(models, model_names, X_test, y_test, ax=ax)

fig.tight_layout()

## Hyperopt for XGBoost Model

In [ ]:
# XGBoost with Hyperopt
if hp_tune:

    xgb_space = {
        "max_depth": hp.choice("max_depth", [2, 4, 8, 16, 32, 64, 128, None]),
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.2),
        "n_estimators": hp.choice("n_estimators", [10, 50, 100, 200, 500]),
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
        "gamma": hp.uniform("gamma", 0, 1),
        "min_child_weight": hp.choice("min_child_weight", [2, 4, 8, 16, 32, 64, 128]),
        "subsample": hp.uniform("subsample", 0.5, 1),
    }

    best_score = 0
    def objective_xgb(params):
        global best_score

        model = xgb.XGBClassifier(
            **params,
            objective = 'binary:logistic',
            seed = random_state,
            n_jobs=-1,
            #eval_metric = 'auc',
        )
        
        score = cross_val_score(model, X_train, y_train, cv=5, scoring="roc_auc").mean()

        #model.fit(X_train_sub, y_train_sub, eval_set=[(X_val, y_val)], verbose=0)
        #model.fit(X_train, y_train)
        #score = roc_auc_score(y_test, model.predict_proba(X_test)[:, model.classes_ == 1])

        # print(f"Score: {score}", params)

        # if score > best_score:
        #     best_score = score
        #     print(f"New best score: {best_score}")
        #     print(params)
            
        return {"loss": 1 - score, "status": STATUS_OK}

    trials_xgb = Trials()

    best_xgb = fmin(
        fn=objective_xgb,
        space=xgb_space,
        algo=tpe.suggest,
        max_evals=100,
        trials=trials_xgb,
        verbose=2,
    )

    print(best_xgb)

In [ ]:
p = {'colsample_bytree': 0.5261530316412195, 'gamma': 0.4666423706737293, 'learning_rate': 0.043284110571295886, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 4, 'subsample': 0.8142370483321494}
model = xgb.XGBClassifier(
    **p,
    objective = 'binary:logistic',
    seed = random_state,
    n_jobs=-1,
    #eval_metric = 'auc',
)

#score = cross_val_score(model, X_train, y_train, cv=5, scoring="roc_auc").mean()

model.fit(X_train, y_train)
roc_auc_score(y_test, model.predict_proba(X_test)[:, model.classes_ == 1])

## Create Submission

In [ ]:
full_data["is_discount"].value_counts()

In [ ]:
final_classifier = AverageClassifier([voting_alt_2, stacking_classifier_2])

X_full = full_data.drop(columns=["conversion", "ROW_ID"])
X_full = X_full.select_dtypes(include='number')

y_full = full_data["conversion"]

final_classifier.fit(X_full, y_full)

# Should be 1, full_data contains test_data 
roc_auc_score(y_test, final_classifier.predict_proba(X_test)[:, final_classifier.classes_ == 1])

In [ ]:
X_test.columns

In [ ]:
roc_auc_score(y_full, final_classifier.predict_proba(X_full)[:, final_classifier.classes_ == 1])

In [ ]:
from sklearn.utils._pprint import _EstimatorPrettyPrinter

In [ ]:
import sklearn

In [ ]:
names_dif = set(X_full.columns)

for model in models:
    # if Pipeline, get last step
    if isinstance(model, sklearn.pipeline.Pipeline):
        model = model.steps[-1][1]
    
    # print feature names
    if isinstance(model, xgb.XGBClassifier):
        print(model.get_booster().feature_names)
    elif isinstance(model, VotingClassifier):
        for estimator in model.estimators_:
            if isinstance(estimator, sklearn.pipeline.Pipeline):
                estimator = estimator.steps[-1][1]
            print(estimator)
            if isinstance(estimator, xgb.XGBClassifier):
                print(estimator.get_booster().feature_names)
            #elif isinstance(estimator, HistGradientBoostingClassifier):
             #   print(estimator.feature_names_)
            #elif isinstance(estimator, GradientBoostingClassifier):
             #   print(estimator.feature_names_)
    elif isinstance(model, StackingClassifier):
        for estimator in model.estimators_:
            if isinstance(estimator, sklearn.pipeline.Pipeline):
                estimator = estimator.steps[-1][1]
            print(estimator)
            if isinstance(estimator, xgb.XGBClassifier):
                print(estimator.get_booster().feature_names)
            #elif isinstance(estimator, HistGradientBoostingClassifier):
             #   print(estimator.feature_names_)
            #elif isinstance(estimator, GradientBoostingClassifier):
              #  print(estimator.feature_names_)
    #elif isinstance(model, BaggingClassifier):
    #    print(model.base_estimator_.get_booster().feature_names)
    elif isinstance(model, HistGradientBoostingClassifier):
        #print(model.feature_names_)
        pass
    elif isinstance(model, GradientBoostingClassifier):
        # print(model.feature_names_)
        pass
    elif isinstance(model, RandomForestClassifier):
       pass



In [ ]:
feature_cols

In [546]:
feature_cols = xgb4_model.get_booster().feature_names

model = final_classifier

eval_data = comp_data[comp_data["ROW_ID"].notna()]
# del comp_data

# Predict on the evaluation set
eval_data = eval_data.drop(columns=["conversion"])
eval_data = eval_data.select_dtypes(include='number')
y_preds = model.predict_proba(eval_data[feature_cols])[:, model.classes_ == 1].squeeze()

# Make the submission file
submission_df = pd.DataFrame({"ROW_ID": eval_data["ROW_ID"], "conversion": y_preds})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("los_simuladores_avg(stack(bagging_classifier_xgb(xgb), xgb, hgb, abc), voting(xgb, hgb, abc))_full_data.csv", index=False)

#del eval_data
#del submission_df
#gc.collect()

In [547]:
# Save model
import pickle

pickle.dump(final_classifier, open("los_simuladores_avg(stack(bagging_classifier_xgb(xgb), xgb, hgb, abc), voting(xgb, hgb, abc))_full_data.pkl", "wb"))